## Does R&D Spending Improve Revenue for Companies?

In [1]:
#Packages
import pandas as pd
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler
import statsmodels.api as sm
from sklearn.cluster import KMeans

### Data Preparation

#### Pull Data - API

In [2]:
# Header to pull API
headers = {'User-Agent': 'DSC680-project'}

In [3]:
# Needed data
metrics = {'Revenue': 'Revenues',
           'R&D': 'ResearchAndDevelopmentExpense',
           'Net_Income': 'NetIncomeLoss'}

In [4]:
# Range
years = range(2008, 2023) 

In [5]:
# Empty dictionary
data = {}

In [6]:
# For loop to download each metric
for label, metric in metrics.items():
    # Empty list for each row
    rows = []
    
    
    # For loop to download each year for each company
    for year in years:
        # API - Change link for each matric and year
        url = f'https://data.sec.gov/api/xbrl/frames/us-gaap/{metric}/USD/CY{year}.json'
        # Request API
        response = requests.get(url, headers=headers)
        
        
        # Make sure request worked
        if response.status_code == 200:
            # Make webdata into dict
            data_pull = response.json()
            # Pull general account info for each company
            data_pull = data_pull['data']
            
            
            # For loop to store each company for each year
            for entry in data_pull:
                # Store only the wanted data
                rows.append({
                    # Company ID
                    'Company_ID': entry.get('cik'),
                    # Company Name
                    'Company_Name': entry.get('entityName'),
                    # Year
                    'Year': year,
                    # Metrics
                    label: entry.get('val'),
                    # Date it was filled
                    'Filing_Date': entry.get('end')})
        else:
            # Print Failer notice if it does not work
            print("Failed to retrieve")
    # Add
    time.sleep(0.15)
    
    # Make all metrics into data frames
    data[label] = pd.DataFrame(rows)

In [7]:
# Merge all three metrics into one dataframe
data_1 = data['Revenue'].merge(
    data['R&D'], on=['Company_ID', 'Company_Name', 'Year', 'Filing_Date'], 
    how='outer').merge(data['Net_Income'], on=['Company_ID', 'Company_Name', 
                                               'Year', 'Filing_Date'], how='outer')

In [8]:
data_1

,Company_ID,Company_Name,Year,Revenue,Filing_Date,R&D,Net_Income
0,864328,BJ SERVICES CO,2008,5.359077e+09,2008-09-30,7.199700e+07,6.093650e+08
1,64978,MERCK SHARP & DOHME CORP.,2008,2.385030e+10,2008-12-31,4.805300e+09,7.808400e+09
2,1335793,CNX GAS CORP,2008,7.894210e+08,2008-12-31,NaN,2.390730e+08
3,868809,XTO ENERGY INC,2008,7.695000e+09,2008-12-31,NaN,1.912000e+09
4,1094316,TRINTECH GROUP PLC,2008,3.966400e+07,2009-01-31,6.069000e+06,-1.232000e+06
...,...,...,...,...,...,...,...
100402,2025410,"StandardAero, Inc.",2022,NaN,2022-12-31,NaN,-2.100000e+07
100403,2037804,New Mountain Private Credit Fund,2022,NaN,2022-12-31,NaN,4.987100e+07
100404,2040127,KARMAN HOLDINGS INC.,2022,NaN,2022-12-31,NaN,-1.409862e+07
100405,2042694,Primo Brands Corp,2022,NaN,2022-12-31,NaN,-1.267000e+08


#### Clean Data

In [9]:
# Delete and duplicates
data_1 = data_1.drop_duplicates()

In [10]:
# Remove any Revenue that is 0 to predict sales growth.
# Do not know for sure that NaN means they did not spend money 
data_2 = data_1.dropna(subset=['Revenue', 'Net_Income', 'R&D'])

Notes: Do not know for sure that NaN means no money was spent on R&D. After looking at SEC data it does not seemto be required. So NaN was dropped.

In [11]:
data_2

,Company_ID,Company_Name,Year,Revenue,Filing_Date,R&D,Net_Income
0,864328,BJ SERVICES CO,2008,5.359077e+09,2008-09-30,7.199700e+07,6.093650e+08
1,64978,MERCK SHARP & DOHME CORP.,2008,2.385030e+10,2008-12-31,4.805300e+09,7.808400e+09
4,1094316,TRINTECH GROUP PLC,2008,3.966400e+07,2009-01-31,6.069000e+06,-1.232000e+06
10,890801,"MCAFEE, INC.",2008,1.600065e+09,2008-12-31,2.520200e+08,1.722090e+08
12,758004,NOVELL INC,2008,9.565130e+08,2008-10-31,1.915470e+08,-8.745000e+06
...,...,...,...,...,...,...,...
45908,1990145,Holdco Nuvo Group D.G Ltd.,2022,0.000000e+00,2022-12-31,9.893000e+06,-2.067900e+07
45911,1991592,INLIF LIMITED,2022,6.652308e+06,2022-12-31,5.047110e+05,5.375550e+05
45913,1993727,SENSTAR TECHNOLOGIES CORPORATION,2022,3.555800e+07,2022-12-31,4.032000e+06,3.831000e+06
45915,1996862,BUNGE GLOBAL SA,2022,6.723200e+10,2022-12-31,3.300000e+07,1.610000e+09


Notes: Large part of the data is NA. Also pull the industry because not all companies need R&D

In [12]:
# Make copy of data due to error
data_3 =  data_2.copy()

#### Unique CIKs - Pull data API

In [13]:
# Collect CIKs as a unique list
ciks = data_3['Company_ID'].astype(str).drop_duplicates().tolist()

# Empty list for each row
ciks_rows = []

In [ ]:
# Need CIKs code to be able to find industry
# For loop to pull each ID
for cik in ciks:
    # Add zeros before Company ID to follow SEC format
    cik_zero = cik.zfill(10)
    #link for each company ID
    url = f'https://data.sec.gov/submissions/CIK{cik_zero}.json'
    # Header
    headers = {'User-Agent': 'DSC680-project'}  
    # Request data
    response = requests.get(url, headers=headers)
    
    
    # If everything is good, pull the wanted information 
    if response.status_code == 200:
        # Make the files readable
        ciks_data = response.json()
        ciks_rows.append({
            # Company ID, need to merge
            'Company_ID': cik.lstrip('0'),
            # Classification code for SEC
            'SIC': ciks_data.get('sic'),
            # More Specific than Industry
            'SIC_Description': ciks_data.get('sicDescription'),
            # Pull Industry
            'Industry': ciks_data.get('ownerOrg'),
            # More company size (Not sure if needed)
            'Category': ciks_data.get('category')})
    # Skip company if it does not have it
    else:
        continue
    # Prevent error
    time.sleep(0.10)

In [ ]:
# Turn into DataFrame
ciks_data = pd.DataFrame(ciks_rows)

In [ ]:
ciks_data

In [ ]:
# Make sure both Company_ID is a str
data_3['Company_ID'] = data_3['Company_ID'].astype(str)
ciks_data['Company_ID'] = ciks_data['Company_ID'].astype(str)

In [ ]:
# Merge on Company_ID
data_4 = pd.merge(data_3, ciks_data, on='Company_ID', how='left')

In [ ]:
data_4

In [ ]:
print(data_4.isna().sum())

Notes: A lot of Industry are missing. Add Industry. No file found. Make a table. Links below to find information

### Make Industry Dataset

In [ ]:
# SIC Division
sic_divisions = [
    {"Division": "A", "Name": "Agriculture, Forestry, and Fishing", "Start": 100, "End": 999},
    {"Division": "B", "Name": "Mining", "Start": 1000, "End": 1499},
    {"Division": "C", "Name": "Construction", "Start": 1500, "End": 1799},
    {"Division": "D", "Name": "Manufacturing", "Start": 2000, "End": 3999},
    {"Division": "E", "Name": "Transportation, Communications, Electric, Gas & Sanitary", "Start": 4000, "End": 4999},
    {"Division": "F", "Name": "Wholesale Trade", "Start": 5000, "End": 5199},
    {"Division": "G", "Name": "Retail Trade", "Start": 5200, "End": 5999},
    {"Division": "H", "Name": "Finance, Insurance & Real Estate", "Start": 6000, "End": 6799},
    {"Division": "I", "Name": "Services", "Start": 7000, "End": 8999},
    {"Division": "J", "Name": "Public Administration", "Start": 9100, "End": 9729},
    {"Division": "-", "Name": "Nonclassifiable Establishments", "Start": 9900, "End": 9999}]

Links: <br>
https://www.naics.com/sic-codes-industry-drilldown/, <br>
https://siccode.com/sic-code-lookup-directory, <br>
https://fieldtexcases.com/blog/manufacturing-sic-codes/?utm_source=chatgpt.com

In [ ]:
# Make a funcation to make a table of all codes
def get_division(sic):
    for d in sic_divisions:
        #only make a list between the SIC cat. codes
        if d["Start"] <= sic <= d["End"]:
            # Return results
            return d["Division"], d["Name"]
    # Give no result if it does not meet the requirements
    return None, None

In [ ]:
# Funcation to make the numbers from get_division into int format 
def int_division(x):
    try:
        # Make results an int.
        x_int = int(x)
        # Return results as a pd series for dataframe
        return pd.Series(get_division(x_int))
    # If error occurs enter none
    except Exception:
        return pd.Series([None, None])

In [ ]:
data_4[["SIC_Division", "SIC_Industry"]] = data_4["SIC"].apply(int_division)

In [ ]:
data_4.head()

#### Clean Data

In [ ]:
# Calculate the sales growth
# sort to make the results sort each year for each company
data_5 = data_4.sort_values(['Company_ID', 'Year'])
# Make new column for sales growth for revenue
data_5['Sales_Growth'] = data_4.groupby('Company_ID')['Revenue'].pct_change()
# Make new column for sales growth for R&D
data_5['RD_Growth'] = data_4.groupby('Company_ID')['R&D'].pct_change()

In [ ]:
# Check for NaN
print(data_5.isna().sum())

In [ ]:
# Drop NaN
data_5 = data_5.dropna(subset=['Industry','SIC_Division', 'SIC_Industry', 'Sales_Growth'])

### Do companies that currently spend heavily on R&D achieve faster sales growth than companies spending less?

In [ ]:
# Group by industry and calculate average sales growth
industry_growth = data_5.groupby('SIC_Industry')['R&D'].mean().sort_values(ascending=False)

##### Average Sales Growth by Industry

In [ ]:
# Graph size
plt.figure(figsize=(10, 6))
# Make a bar chart
industry_growth.plot(kind='bar')
plt.title('Average Sales Growth by Industry')
# Make industry easy to read
plt.xticks(rotation=45, ha='right')
# Make labels print within size
plt.tight_layout()
# Print graph
plt.show()

In [ ]:
# Fix error. Remove outliers
# 25% of the data Q1
Q1 = data_5['Sales_Growth'].quantile(0.25)
# 75% of the data Q3
Q3 = data_5['Sales_Growth'].quantile(0.75)
# Find the spread (IQR)
spread = Q3 - Q1

In [ ]:
# find lower outliers
lower_bound = Q1 - 1.5 * spread
# Find upper outliers
upper_bound = Q3 + 1.5 * spread

In [ ]:
# Filter out outliers
data_6 = data_5[(data_5['Sales_Growth'] >= lower_bound) & (data_5['Sales_Growth'] <= upper_bound)]

##### R&D Spending Vs Sales Growth - Scatterplot

In [ ]:
# Figure Size
plt.figure(figsize=(5,5))
# Make a scatter plot
sns.scatterplot(x='R&D', y='Sales_Growth', data=data_6)
# Print graph
plt.show()

Notes: Try Random forest. results are not linear

 ##### Random Forest Regressor

In [ ]:
# Make copy to prevent error
data_rnd = data_6.copy()

In [ ]:
# Define features and targets to predict if current sales growth
X = data_rnd[['R&D']]
y = data_rnd['Sales_Growth']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Allow random forest
rf_model = RandomForestRegressor(random_state=42)

In [ ]:
# Fit model
rf_model.fit(X_train, y_train)

In [ ]:
# Predict model
y_pred_rf = rf_model.predict(X_test)

In [ ]:
print("Random Forest - Just R&D")
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R2:", r2_score(y_test, y_pred_rf))

In [ ]:
# Define features and targets to predict if current sales growth
# include year to account for covid and industry due
X = pd.get_dummies(data_rnd[['R&D', 'Year', 'SIC_Industry']])
y = data_rnd['Sales_Growth']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Allow random forest
rf_model = RandomForestRegressor(random_state=42)

In [ ]:
# Fit model
rf_model.fit(X_train, y_train)

In [ ]:
# Predict model
y_pred_rf = rf_model.predict(X_test)

In [ ]:
print("Random Forest - R&D, Year, SIC_Industry")
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R2:", r2_score(y_test, y_pred_rf))

Not a good model. Try robust scaler for R&D and sales growth to decrease skew

In [ ]:
# Copy of data for scaled data
data_7 = data_6.copy()

In [ ]:
# Allow scaler
scaler = RobustScaler()

In [ ]:
# Scale R&D and sales growth
data_7['R&D_scaled'] = scaler.fit_transform(data_7[['R&D']])
data_7['Sales_Growth_scaled'] = scaler.fit_transform(data_7[['Sales_Growth']])

In [ ]:
# Define features and targets to predict
X = pd.get_dummies(data_7[['R&D_scaled', 'Year', 'SIC_Industry']])
y = data_7['Sales_Growth_scaled']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit model
rf_model.fit(X_train, y_train)

In [ ]:
# Predict model
y_pred_rf = rf_model.predict(X_test)

In [ ]:
print("Random Forest")
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R2:", r2_score(y_test, y_pred_rf))

Model is even worse. Only look at the top three

In [ ]:
# Top three sales growth
keep = ['Agriculture, Forestry, and Fishing','Manufacturing','Services']

In [ ]:
# New data set of only top three
data_8 = data_7[data_7['SIC_Industry'].isin(keep)]

In [ ]:
# Make copy to prevent error
data_rnd_top3 = data_8.copy()

In [ ]:
# Define features and targets to predict
X = data_rnd_top3[['R&D']]
y = data_rnd_top3['Sales_Growth']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit model
rf_model.fit(X_train, y_train)

In [ ]:
# Predict model
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# Notes: Top three
print("Random Forest")
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R2:", r2_score(y_test, y_pred_rf))

Try a different model

Random Forest - Only looking at R&D as Indep.
MAE: 0.21793900803113708
RMSE: 0.28644902589828736
RMSE: 0.28644902589828736
R2: -0.23215625461103118


Random Forest - Three indep. varb
MAE: 0.2049755338401916
RMSE: 0.27509560160027885
R2: -0.1364187217596715

##### Gradient Boosting

In [ ]:
# Define features and targets to predict 
X = data_7[['R&D']]
y = data_7['Sales_Growth']

In [ ]:
# Allow Gradient Boosting
gb_model = GradientBoostingRegressor(random_state=42)

In [ ]:
# Fit the model
gb_model.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_gb = gb_model.predict(X_test)

In [ ]:
print("Gradient Boosting - R&D")
print("MAE:", mean_absolute_error(y_test, y_pred_gb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("R2:", r2_score(y_test, y_pred_gb))

In [ ]:
# Define features and targets to predict 
X = pd.get_dummies(data_rnd[['R&D', 'Year', 'SIC_Industry']])
y = data_rnd['Sales_Growth']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit the model
gb_model.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_gb = gb_model.predict(X_test)

In [ ]:
print("Gradient Boosting - Top Three with Sales Growth")
print("MAE:", mean_absolute_error(y_test, y_pred_gb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("R2:", r2_score(y_test, y_pred_gb))

Notes: Model is better. Look at Inustry and Year

In [ ]:
# Define features and targets to predict. Look at all
X = pd.get_dummies(data_7[['R&D', 'Year', 'SIC_Industry']])
y = data_7['Sales_Growth']

In [ ]:
# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit model
gb_model.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_gb = gb_model.predict(X_test)

In [ ]:
print("Gradient Boosting - All Industry")
print("MAE:", mean_absolute_error(y_test, y_pred_gb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gb)))
print("R2:", r2_score(y_test, y_pred_gb))

Model is the same. Does not look like it made any difference to look at the top three or all of them.

R&D, year, and industry only explained a small amount of variation in sales growth

#### Actual Sales Growth Vs. Predicted

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Make scatter plot
plt.scatter(y_test, y_pred_gb, alpha=0.6)
# X-label
plt.xlabel('Actual Sales Growth')
# Y-label
plt.ylabel('Predicted Sales Growth')
# Plot graph
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red')
# Print graph
plt.show()

#### Test Set Actual vs Predicted Sales Growth - Scatterplot

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Scatterplot of Actual Data
plt.scatter(X_test['R&D'], y_test, label='Actual', alpha=0.6)
#Scatterplot of Predicted data
plt.scatter(X_test['R&D'], y_pred_gb, label='Predicted', alpha=0.6, color='red')
# X- Label
plt.xlabel('R&D')
#Y-Label
plt.ylabel('Sales Growth)')
# Titel
plt.title('Test Set: Actual vs Predicted Sales Growth by R&D')
#Key
plt.legend()
#Print Graph
plt.show()

### When a company increases its R&D spending in one year, does its sales growth accelerate in the subsequent year?

Notes: Not enough data per company for AIRMA. Use OLS

In [ ]:
#Make copy of data
data_9 = data_8.copy()

In [ ]:
# Group by Company ID and R&D. Shift one year
data_9['R&D'] = data_9.groupby('Company_ID')['R&D'].shift(1)

In [ ]:
# Drop Na
data_9 = data_9[['Revenue', 'R&D', 'SIC_Industry']].dropna()

In [ ]:
# Revenue ~ Prior year R&D
# Independ. Varb.
X = data_9[['R&D']]
# Add intercept
X = sm.add_constant(X)
# Depend. Varb.
y = data_9['Revenue']

In [ ]:
# Fit Model
model = sm.OLS(y, X).fit()

In [ ]:
print(model.summary())

Notes:
- R&D coefficient = 10.15 | significant because p value is also p < .001
- R²= 0.274 | means 27.4% of revenue variation explained by prior year’s R&D
- Each 1 million increase in prior year’s R&D spending is associated with an average increase of about 10.15 million in revenue the following year.
- The relationship is statistically significant (p < 0.001), suggesting a real effect.
- R² = 0.27: Prior year R&D explains a portion of future revenue, but there are still many other factors.

### Classify companies into distinct groups, e.g., high investors, moderate investors, and low investors, based on their R&D spending and sales growth patterns.

In [ ]:
# Group companies by mean R&D and Sales Growth
data_k = data_8.groupby('Company_ID').agg({'R&D': 'mean', 
                                         'Sales_Growth': 'mean'}).dropna()

In [ ]:
# Scale, fit and transform the 
X_optK = StandardScaler().fit_transform(data_k[['R&D', 'Sales_Growth']])

In [ ]:
# Empty list for k clusters
k_clus = []

##### Elbow Graph - Find K-Cluster

In [ ]:
# Find Optimal Clster
for k in range(1, 10):
    # For each cluster fit a Kmeans model
    km = KMeans(n_clusters=k, random_state=42).fit(X_optK)
    # Append list
    k_clus.append(km.inertia_)

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Plot the k clusters and mark each one
plt.plot(range(1,10), k_clus, marker='o')
# Title
plt.title('Elbow Method for Optimal K')
# Print Graph
plt.show()

In [ ]:
# Allow KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)

In [ ]:
# Data clusters to data table
data_k['Cluster'] = kmeans.fit_predict(data_k[['R&D', 'Sales_Growth']])

In [ ]:
data_k.head(5)

In [ ]:
data_k['Cluster'].unique()

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Make a box blot of the clusters (Sales Growth)
sns.boxplot(x='Cluster', y='Sales_Growth', data=data_k)
# Print graph
plt.show()

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Make a box blot of the clusters (R&D)
sns.boxplot(x='Cluster', y='R&D', data=data_k)
# Print Graph
plt.show()

In [ ]:
# Figure size
plt.figure(figsize=(7, 5))
# Scatter Plot of all the clusters
sns.scatterplot(x='R&D', y='Sales_Growth', hue='Cluster', data=data_k, palette='tab10')
# Print Graph
plt.show()